In [18]:
import numpy as np
import math
from scipy.special import iti0k0, k0
from scipy.integrate import quad
from scipy.integrate import fixed_quad
from fwell.ffunc.frac import matr_pd_frac_nnnn
from fwell.ffunc.aux import eulsum_v, sexp

In [2]:
# Sample well for testing purposes
from fwell.fwell import FWell
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

outer_bound = "nnnn"
top_bound = "imp"
bottom_bound = 'imp'
wtype = 'frac'
nseg = 20
nwells = 1
Xe = 1000
Ye = 1000
Xf = 100
xw = Xe/2
yw = Ye/2
xwds = [xw/Xf]
ywds = [yw/Xf]
xed = Xe/Xf
yed = Ye/Xf
Fcd = 10
k = 10
h = 10
ct = 1e-5
mu = 1
B = 1
fi = 0.1
Q = 100
ts = np.logspace(-2,3, 100)
CT = 0.00036*k/(fi*mu*ct*Xf*Xf)
CP = Q*18.42*mu*B/(k*h)
tds = ts*CT
fwell = FWell(outer_bound, top_bound, bottom_bound, wtype, nseg, nwells, xwds, ywds, xed=xed, yed=yed, attrs={'Fcd': Fcd})
xd, xwd, yd, ywd, x1, x2 = fwell.lw.xd, fwell.lw.xwd, fwell.lw.yd, fwell.lw.ywd, fwell.lw.x1, fwell.lw.x2
zd = 0.501 * np.ones_like(x1)
zwd = 0.5 * np.ones_like(x1)
hd = 0.1
buf = fwell.lw.buf

In [3]:
def funsum(func, first_n = 1, blksize = 10, rtol = 1e-18, MAXITER = 10000):
    # naive algo for series summation
    # first_n - number of the first term in series, may be 0 or 1
    # func - lambda-like function that returns matrix and is a member of a series that depends on n
    # blksize - number of terms in series used to calculate incremental sums
    # rtol - relative tolerance of the summation: if abs(sum(func(n), n, n+blksize))<rtol*abs(sum(func(n), 1, n)) then stop
    # MAXITER - maximum number of summation steps. Total count of members used in sum is therefore blksize*MAXITER
    fsum = 0
    for j in range(0, MAXITER):
        dum = 0
        rng = np.arange(j*blksize+first_n, (j+1)*blksize+first_n)
        for n in rng:
            dum += func(n)
        fsum += dum
        if np.all(np.abs(dum)<=rtol*np.abs(fsum)):
            return fsum, j
    raise RuntimeError("funsum did not converge with {} members".format(MAXITER*blksize))

In [4]:
# test funsum
from fwell.ffunc.ffuncs import iF2Ek, iF2E
u = 0.1
# built-in
rep_ans = iF2E(x1, x2, u, xd, xwd, xed, xed, 1, yd, ywd, yed, buf, "if2e")
# funsum
func = lambda n: iF2Ek(n, buf, x1, x2, u, xd, xwd, xed, xed, 1, yd, ywd, yed, "if2e1")
test_ans = funsum(func)
print(np.allclose(rep_ans, test_ans[0]))
print(test_ans[1])

True
2


In [5]:
def IFhor():
    return IFrac(u, x1, x2, xd, xwd, xed, yd, ywd, yed, buf, id_frac1, id_frac2) +\
            IF1(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed) + IF2()

In [6]:
def IFrac(u, x1, x2, xd, xwd, xed, yd, ywd, yed, buf, id1, id2):
    return matr_pd_frac_nnnn(u, x1, x2, xd, xwd, xed, yd, ywd, yed, buf, id1, id2)

In [7]:
def IF1(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed):
    ans = IF11(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed) 
    ans += IF12(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed) 
    ans += IF13(u, x1, x2, xed, zd, zwd, hd, yd, ywd)

In [8]:
def IF11(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed):
    func = lambda n: IF11_n(n, u, zd, zwd, hd, yd, ywd, yed)
    return np.pi*(x2-x1)/xed*funsum(func)[0]

def IF11_n(n, u, zd, zwd, hd, yd, ywd, yed):
    pi = np.pi
    dum = n*pi/hd
    en = np.sqrt(u+dum*dum)
    ans = 1/en*np.cos(n*pi*zd/hd)*np.cos(n*pi*zwd/hd)
    ans *= 1 + sexp(en, yed)
    ans *= np.exp(-en*(yd+ywd)) + np.exp(-en*(2*yed-yd-ywd)) + np.exp(-en*(2*yed-np.abs(yd-ywd)))
    return ans

In [9]:
def IF12(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed):
    func = lambda n: IF12_n(n, u, zd, zwd, hd, yd, ywd, yed)
    return np.pi*(x2-x1)/xed*funsum(func)[0]

def IF12_n(n, u, zd, zwd, hd, yd, ywd, yed):
    pi = np.pi
    dum = n*pi/hd
    en = np.sqrt(u+dum*dum)
    ans = 1/en*np.cos(n*pi*zd/hd)*np.cos(n*pi*zwd/hd)*np.exp(-en*np.abs(yd-ywd))*sexp(en, yed)
    return ans

In [10]:
def IF13(u, x1, x2, xed, zd, zwd, hd, yd, ywd):
    func = lambda n: IF13_n(n, u, zd, zwd, hd, yd, ywd)
    su = np.sqrt(u)
    fsum = 0
    for s1 in [-1, 1]:
        fsum += k0(np.sqrt(np.square(zd+s1*zwd)+np.square(yd-ywd))*np.sqrt(u)) # first two terms in series with n == 0
    fsum += funsum(func)[0]
    return 0.5*(x2-x1)/xed*(hd*fsum - np.pi/su*np.exp(-su*np.abs(yd-ywd)))

def IF13_n(n, u, zd, zwd, hd, yd, ywd):
    fsum = 0
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            fsum += k0(np.sqrt(np.square(zd+s1*zwd+s2*2*n*hd)+np.square(yd-ywd))*np.sqrt(u))
    return fsum

In [11]:
# testing functions
u = 0.1
%timeit -n10 -r10 IF11(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed)
%timeit -n10 -r10 IF12(u, x1, x2, xed, zd, zwd, hd, yd, ywd, yed)
%timeit -n10 -r10 IF13(u, x1, x2, xed, zd, zwd, hd, yd, ywd)

3.74 ms ± 75.3 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
2.53 ms ± 30.4 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
413 ms ± 303 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [12]:
def IF2():
    return IF21(u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed) + IF22()

In [13]:
def IF21(u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed):
    func = lambda n: IF21_n(n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed)
    return 2*np.pi/xed*funsum(func)[0]

def IF21_n(n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed):
    func = lambda k: IF21_n_k(k, n, u, x1, x2, xd, xwd, xed, yd, yed, hd)
    pi = np.pi
    return np.cos(n*pi*zd/hd)*np.cos(n*pi*zwd/hd)*funsum(func)[0]

def IF21_n_k(k, n, u, x1, x2, xd, xwd, xed, yd, yed, hd):
    pi = np.pi
    ekn = np.sqrt(u+n*n*pi*pi/hd/hd+k*k*pi*pi/xed/xed)
    se = sexp(ekn, yed)
    ans = 2*xed/k/ekn/pi*np.cos(k*pi*xd/xed)*np.sin(0.5*k*pi/xed*(x2-x1))*np.cos(0.5*k*pi/xed*(2*xwd+x1+x2))
    ans *= (np.exp(-ekn*(yd+ywd))+np.exp(-ekn*(2*yed-yd-ywd))+np.exp(-ekn*(2*yed-np.abs(yd-ywd))))*(1+se) +\
            np.exp(-ekn*np.abs(yd-ywd))*se
    return ans

In [15]:
def IF22():
    return IF221() - IF222()

def IF221(u, x1, x2, axd, axwd, axed, azd, azwd, ahd, ayd, aywd, ayed):
    mshape = x1.shape
    assert x2.shape == mshape
    ans = np.zeros(mshape, dtype = np.float)
    # here should be shapes and types check!
    
    inds_dyd0 = np.isclose(yd, ywd)
    inds_dyd_nnz = np.logical_not(inds_dyd0)
    if np.any(inds_dyd_nnz):
        x1_ = x1[inds_dyd_nnz]
        x2_ = x2[inds_dyd_nnz]
        xd_ = xd[inds_dyd_nnz]
        xwd_ = xwd[inds_dyd_nnz]
        xed_ = xed[inds_dyd_nnz]
        zd_ = zd[inds_dyd_nnz]
        zwd_ = zwd[inds_dyd_nnz]
        hd_ = hd[inds_dyd_nnz]
        yd_ = yd[inds_dyd_nnz]
        ywd_ = ywd[inds_dyd_nnz]
        yed_ = yed[inds_dyd_nnz]
        func = lambda n: IF221_n_dyd_nnz(n, u, x1_, x2_, xd_, xwd_, xed_, zd_, zwd_, hd_, yd_, ywd_, yed_)
        ans[inds_dyd_nnz] = funsum(func)[0]
    if np.any(inds_dyd0):
        x1_ = x1[inds_dyd0]
        x2_ = x2[inds_dyd0]
        xd_ = xd[inds_dyd0]
        xwd_ = xwd[inds_dyd0]
        xed_ = xed[inds_dyd0]
        zd_ = zd[inds_dyd0]
        zwd_ = zwd[inds_dyd0]
        hd_ = hd[inds_dyd0]
        yd_ = yd[inds_dyd0]
        ywd_ = ywd[inds_dyd0]
        yed_ = yed[inds_dyd0]
        func = lambda n: IF221_n_dyd0(n, u, x1_, x2_, xd_, xwd_, xed_, zd_, zwd_, hd_, yd_, ywd_, yed_)[0]
        fsum = funsum(func)[0]
        fsum += ftrick(u, x1_, x2_, xd_, xwd_, xed_, zd_, zwd_, hd_, yd_, ywd_, yed_)
        ans[inds_dyd0] = fsum
    return ans

def IF221_n_dyd0(n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed):
    fsum = 0
    for s1 in [-1, 1]:
        t1 = xd + s1*xwd - x2
        t2 = xd + s1*xwd - x1
        sign_t1 = -1 + 2*(t1 > 0)
        sign_t2 = -1 +2*(t2 > 0)
        fsum += ki(np.sqrt(u+n*n*pi*pi/hd/hd)*np.abs(t1))*sign_t1
        fsum += ki(np.sqrt(u+n*n*pi*pi/hd/hd)*np.abs(t2))*sign_t2
    func = lambda k: IF221_n_k_dyd0(k, n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed)
    fsum += funsum(func)[0]
    ans = np.cos(n*pi*zd/hd)*np.cos(n*pi*zwd/hd)*fsum
    return ans, kmax

def IF221_n_k_dyd0(k, n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed):
    fsum = 0
    for s1 in [-1, 1]:
        for s2 in [-1, 1]:
            t1 = xd + s1*xwd - x2 + 2*k*xed
            t2 = xd + s1*xwd - x1 + 2*k*xed
            sign_t1 = -1 + 2*(t1 > 0)
            sign_t2 = -1 + 2*(t2 > 0)
            fsum += ki(np.sqrt(u+n*n*pi*pi/hd/hd)*np.abs(t1))*sign_t1
            fsum += ki(np.sqrt(u+n*n*pi*pi/hd/hd)*np.abs(t2))*sign_t2
    return fsum
    
def ftrick(u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed):
    # this part is an addition to sum cause by terms where t1 <= 0 and t2 >= 0
    # t1 = (xd+s1*xwd-x2+2*s2*k*xed), t2 = (xd + s1*xwd - x1 + 2*s2*k*xed)
    func = lambda n: IF13_n(n, u, zd, zwd, hd, yd, ywd)
    su = np.sqrt(u)
    fsum = 0
    for s1 in [-1, 1]:
        fsum += k0(np.sqrt(np.square(zd+s1*zwd)+np.square(yd-ywd))*np.sqrt(u)) # first two terms in series with n == 0
    fsum += funsum(func)[0]
    fsum = 0.5*(hd*fsum - np.pi/np.sqrt(u))
    # here we define terms to which we should add the above sum:
    kmins = []
    kmaxs = []
    for s1 in [-1, 1]:
        for s2 in [-1, 1]:
            kmins.append(math.floor(np.min((x1-s1*xwd-xd)/(2*s2*xed))))
            kmaxs.append(math.ceil(np.min((x2-s1*xwd-xd)/(2*s2*xed))))
    indicator = np.zeros_like(x1)
    for s1 in [-1, 1]:
        for s2 in [-1, 1]:
            for k in np.arange(kmin, kmax+1):
                indicator += np.logical_and((xd+s1*xwd-x2+2*s2*k*xed) <= 0, (xd + s1*xwd - x1 + 2*s2*k*xed) >=0)
    return fsum*indicator

def IF221_n_dyd_nnz(n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed):
    fsum = 0
    pi = np.pi
    a = np.sqrt(u+n*n*pi*pi/hd/hd)
    dyd = np.abs(yd-ywd)
    for s1 in [-1, 1]: # account for k==0
        dxd = xd+s1*xwd
        fsum += int_K0(a, dxd, dyd, x1, x2)
    func = lambda k: IF221_n_k(k, n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed)
    fsum += funsum(func)[0]
    return np.cos(n*pi*zd/hd)*np.cos(n*pi*zwd/hd)*fsum



def IF221_n_k(k, n, u, x1, x2, xd, xwd, xed, zd, zwd, hd, yd, ywd, yed):
    pi = np.pi
    a = np.sqrt(u+n*n*pi*pi/hd/hd)
    ans = 0
    dyd = yd-ywd
    for s1 in [-1, 1]:
        for s2 in [-1, 1]:
            dxd = xd + s1*xwd + s2*2*k*xed
            ans += int_K0(a, dxd, dyd, x1, x2)
    return ans

def int_K0(a, adxd, adyd, x1, x2):
    # returns integral from x1 to x2 of function K0(a*sqrt((adxd-x')^2+adyd^2))
    # x1, x2 are matrices with equal shapes - limits of integration
    # if dxd and dyd are matrices, ther shapes should be consistent with x1 and x2 shapes
    # if xdx and dyd are scalars, integrals are calculated for all pairs of limits in x1 and x2 matrices with fixed dxd, dyd
    # a is a scalar
    NDIGITS = 6 # roundoff precision for np.unique function
    mshape = x1.shape
    m = np.zeros(mshape, dtype=np.double) # answer matrix
    # checking shapes of arguments
    assert x2.shape == mshape
    if isinstance(adxd, np.ndarray):
        assert adxd.shape == mshape
        dxd = adxd.copy()
    else:
        dxd = adxd*np.ones(mshape, dtype=np.double)
    if isinstance(adyd, np.ndarray):
        assert adyd.shape == mshape
        dyd = adyd.copy()
    else:
        dyd = adyd*np.ones(mshape, dtype=np.double)
    
    inds_dyd0 = np.isclose(dyd,0.) # for dyd == 0 algorithm is different from != 0
    inds_dyd_nnz = np.logical_not(inds_dyd0)
    
    if np.any(inds_dyd0):
        t1 = dxd[inds_dyd0] - x2[inds_dyd0] # here we account for the sign change when simplify sqrt(x^2) expression
        t2 = dxd[inds_dyd0] - x1[inds_dyd0] #
        mask1 = np.ones_like(t1) - 2*(t1>0)
        mask2 = np.ones_like(t1) - 2*(t2<0)
        t1 = np.round(np.abs(t1), decimals=NDIGITS).flatten()
        t2 = np.round(np.abs(t2), decimals=NDIGITS).flatten()
        nt = len(t1)
        t = np.append(t1,t2)
        ut, inv_t = np.unique(t, return_inverse=True) # we'll calculate integrals only for unique integration limits
        uv = 1./a*iti0k0(ut*a)[1]
        v = uv[inv_t] # inverse-unique operation
        v1 = v[:nt]
        v2 = v[nt:]
        m[inds_dyd0] = v1*mask1+v2*mask2
    if np.any(inds_dyd_nnz):
        t1 = dxd[inds_dyd_nnz] - x2[inds_dyd_nnz] # 
        t2 = dxd[inds_dyd_nnz] - x1[inds_dyd_nnz] #
        dyd = np.round(np.abs(dyd)[inds_dyd_nnz], decimals=NDIGITS).flatten()
        t1 = np.round(t1, decimals=NDIGITS).flatten()
        t2 = np.round(t2, decimals=NDIGITS).flatten()
        t = np.vstack([t1,t2]).T
        t = np.sort(t, axis=1)
        t = np.hstack([t,dyd[np.newaxis].T])
        ut, inv_t = np.unique(t, axis=0, return_inverse=True)
        uv = qgaus(m_bessk0, ut[:,0], ut[:,1], (ut[:,2],a))
        m[inds_dyd_nnz] = uv[inv_t]
    return m

def m_bessk0(x, dyd, a):
    assert a >= 0.
    return k0(a*np.sqrt(np.square(x)+np.square(dyd)))

In [16]:
def IF222(u, x1, x2, zd, zwd, hd, yd, ywd):
    pi = np.pi
    fsum = 0
    su = np.sqrt(u)
    for s1 in [-1, 1]:
        fsum += IF222_n(0, u, s1, -1, zd, zwd, hd, yd, ywd)
        for s2 in [-1, 1]:
            func = lambda n: IF222_n(n, u, s1, s2, zd, zwd, yd, ywd)
            fsum += funsum(func)[0]
    return pi*(x2-x1)/xed*(0.5*hd/pi*fsum - 0.5*np.exp(-su*np.abs(yd-ywd))/su)

def IF222_n(n, u, s1, s2, zd, zwd, hd, yd, ywd):
    return k0(np.sqrt(u)*np.sqrt(np.square(zd + s1*zwd + s2*2*n*hd)+np.square(yd-ywd)))